In [50]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [51]:
print('Importing Librairies...')
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report,r2_score
import warnings
import pickle
warnings.filterwarnings("ignore")
#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print('DEEP LEARNING FOREX V1',Style.RESET_ALL)

#LaDate = input('Date de DL - YYYY-MM-DD ')
LaDate = '2020-01-15'
ticker = 'USDJPY_1H'
indice = 0.1
try:
    os.mkdir('DL_'+ticker+'-'+LaDate)
except:
    pass
try:
    os.mkdir('Boost_'+ticker+'-'+LaDate)
except:
    pass

Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0°
DEEP LEARNING FOREX V1 


In [52]:
def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        try:
            rsi70_list.append(70 - rsi[0])
            rsi30_list.append(rsi[i] - 30)
        except:
            rsi70_list.append(0)
            rsi30_list.append(0)
        
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    df['%Futur'] = ((df['Close'].shift(-1)-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > indice :
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -indice:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
        
    df['target_up'] = up  
    df['target_down'] = down 
    #df = df.dropna()
    
    df = df[['High','Low','Open','Volume','DeltaVol','Close','Varop_Spy','Varhl_spy','RSI',\
             '70 - RSI','RSI - 30','BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
             'delta12_15','delta15_30','delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
             'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon','Boost',\
             '%Futur','target_up','target_down']]
        
    return(df)
    
def deep_learning(df):
    
    seed = 770
    np.random.seed(seed)
    X = df.iloc[:,:-4]
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values
    X.astype(np.float64)
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    model_up = Sequential()
    # Add an input layer 
    model_up.add(Dense(50, activation='relu'))
    # Add one hidden layer 
    model_up.add(Dense(23, activation='relu'))
    # Add an output layer 
    model_up.add(Dense(1, activation='sigmoid'))

    model_down = Sequential()
    # Add an input layer 
    model_down.add(Dense(50, activation='relu'))
    # Add one hidden layer 
    model_down.add(Dense(23, activation='relu'))
    # Add an output layer 
    model_down.add(Dense(1, activation='sigmoid'))

    model_up.compile(loss='binary_crossentropy',
                  optimizer='adam', #rmsprop
                  metrics=['accuracy','mse'])
    
              

    history_up = model_up.fit(Xtrain, ytrain_up,epochs=280, batch_size=8, verbose=0)
    
    model_down.compile(loss='binary_crossentropy',
                  optimizer='adam', #rmsprop
                  metrics=['accuracy','mse'])

    history_down = model_down.fit(Xtrain, ytrain_down,epochs=280, batch_size=8, verbose=0)
    

    train_acc_up = model_up.evaluate(Xtrain, ytrain_up,verbose=1)
    train_acc_down = model_down.evaluate(Xtrain, ytrain_down,verbose=1)

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    score_up = model_up.evaluate(Xtest, ytest_up,verbose=1)
    score_down = model_down.evaluate(Xtest, ytest_down,verbose=1)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    accuracy_up = accuracy_score(ytest_up, yhat_up)
    accuracy_down = accuracy_score(ytest_down, yhat_down)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall_up = recall_score(ytest_up, yhat_up) 
    recall_down = recall_score(ytest_down, yhat_down) 


    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close'].shift(-1)
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,model_up,model_down,scaler)


def save_model(model_up,model_down):
    savename = 'DL_'+ticker+'-'+LaDate+'/Save_'+ticker
    # serialize model to YAML
    model_up_yaml = model_up.to_yaml()
    model_down_yaml = model_down.to_yaml()
    with open(savename+"_up.yaml", "w") as yaml_file:
        yaml_file.write(model_up_yaml)
    with open(savename+"_down.yaml", "w") as yaml_file:
        yaml_file.write(model_down_yaml)
    # serialize weights to HDF5
    model_up.save_weights(savename+"_up.h5")
    model_down.save_weights(savename+"_down.h5")

In [53]:
# PARAMETRES TEMPORELS INITIAUX
start = '2018-05-04'
end = LaDate
print('DATE ORIGINELLE DU DEBUT DE TOUT... :',start)
error = []

        
########################
#### MAIN SKAN55 #######
########################

print(' - Date de début : ',start,' - Date de fin : ',end)
print(Fore.YELLOW,'GENERATION DE LaBase :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
tmps55=time.time()
LaBase = pd.read_csv('USDJPY_1H.csv')
df = LaBase

INDEX = []
for i in range(0,len(df)):
    INDEX.append((str(df.iloc[i,0])[6:10]+'-'+str(df.iloc[i,0])[3:5]+'-'+str(df.iloc[i,0])[:2]))
df.index = INDEX
df = df.drop(['Local time'],axis=1)
df = df[df.index>=start]
delta = round(df.shape[0])
bloc1 = round(delta*0.75)
bloc2 = delta - bloc1

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Base Preparation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
#LaBasePrep.to_csv('LaBasePrep.csv')
df.head()

DATE ORIGINELLE DU DEBUT DE TOUT... : 2018-05-04
 - Date de début :  2018-05-04  - Date de fin :  2020-01-15
 GENERATION DE LaBase :  00:07:00
 Time for complete Base Preparation = 2.960000 seconds 
 


,Open,High,Low,Close,Volume
2018-05-04,109.174,109.215,109.157,109.158,977.5300
2018-05-04,109.158,109.174,109.097,109.104,1038.2100
2018-05-04,109.104,109.234,109.102,109.173,2422.8000
2018-05-04,109.175,109.183,108.992,109.021,2063.5801
2018-05-04,109.021,109.033,108.933,108.985,2375.4800


In [45]:
def boost(df):
    print(Fore.YELLOW,'\nGENERATION DU BOOST :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
    tmps55=time.time()
    
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:bloc1,:-1]
    Xtest = X.iloc[bloc1:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:bloc1]
    ytest = yshift.iloc[bloc1:-1]

    model = xgb.XGBRegressor(n_estimators=200000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=5,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=10, eval_metric=['mae'],eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])
    r2 = r2_score(ytest,y_pred)*100
    df['Boost'] = pred
    return(df,model,r2)


df,model,r2 = boost(df)

print(Fore.GREEN,'\nr2_score = ',r2,'%')

print('% error : ',abs(((df['Boost'].shift(+1)-df['Close'])/df['Close']*100)).mean(),'%')

print('\nEcart min : ',abs(((df['Boost'].shift(+1)-df['Close'])/df['Close']*100)).min(),'%')

print('\nEcart max : ',abs(((df['Boost'].shift(+1)-df['Close'])/df['Close']*100)).max(),'%')

print('\n',model.evals_result())

pickle.dump(model, open('Boost_'+ticker+'-'+LaDate+'/'+ticker+'_boost.dat', "wb"))

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\nTime for complete BOOST GENERATION = %f" %tmps22,'seconds \n',Style.RESET_ALL)

 
GENERATION DU BOOST :  23:41:04
 
r2_score =  98.74338637342316 %
% error :  0.076567833729932 %

Ecart min :  2.3521929559385e-05 %

Ecart max :  2.0051121376559546 %

 {'validation_0': {'mae': [0.88572, 0.162923, 0.094605, 0.092116, 0.092123, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115, 0.092115]}}
 
Time for complete BOOST GENERATION = 6.960000 seconds 
 


In [54]:
###### PREPARATION #########


print('The computing has begun at ',dt.datetime.now().hour,':',dt.datetime.now().minute,':',dt.datetime.now().second)

BasePrep = pd.DataFrame()
tmps55=time.time()

df['DeltaVol'] = df['Volume'] - df['Volume'].shift(1)
df = df.dropna()
df = prepa_data(df)
    
BasePrep = df
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\nTime for complete Preparation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
BasePrep.to_csv('BasePrep.csv')

The computing has begun at  0 : 7 : 7


KeyError: "['Boost'] not in index"

In [47]:
df.shape,df['target_up'].sum(),df['target_down'].sum()

((10653, 36), 652, 676)

In [48]:
df.tail()

,High,Low,Open,Volume,DeltaVol,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,...,lower,var_bollup_kchigh,var_bolllow_kclow,Aroon Up,Aroon Down,Delta Aroon,Boost,%Futur,target_up,target_down
2020-01-17,110.176,110.132,110.161,2539.85,-4323.07,110.140,0.021,0.044,50.204228,0,...,110.095699,-0.099429,0.094463,0.000000,77.777778,-77.777778,110.204773,-0.002724,0,0
2020-01-17,110.155,110.121,110.139,2048.59,-491.26,110.137,0.002,0.034,49.840591,0,...,110.094733,-0.097726,0.086651,55.555556,66.666667,-11.111111,110.204773,0.009988,0,0
2020-01-17,110.149,110.111,110.137,1637.67,-410.92,110.148,-0.011,0.038,51.235320,0,...,110.104881,-0.092830,0.090397,44.444444,55.555556,-11.111111,110.204773,0.004539,0,0
2020-01-17,110.173,110.119,110.150,2267.76,630.09,110.153,-0.003,0.054,51.890157,0,...,110.105245,-0.089840,0.087723,33.333333,44.444444,-11.111111,110.204773,-0.046299,0,0
2020-01-17,110.177,110.102,110.153,2141.02,-126.74,110.102,0.051,0.075,45.219901,0,...,110.105841,-0.087340,0.094642,22.222222,33.333333,-11.111111,110.204773,NaN,0,0


In [49]:
tmps55=time.time()
print(Fore.BLUE,'\nDeeping in blue for ',ticker,Style.RESET_ALL)

resultats,precision_up,precision_down,model_up,model_down,scaler = deep_learning(df)
print(round(time.time()-tmps55,2),' secondes')

if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 1 fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_up,precision_down,model_up,model_down,scaler = deep_learning(df)
    print(round(time.time()-tmps5,2),' secondes')
    
if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 2 fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_up,precision_down,model_up,model_down,scaler = deep_learning(df)
    print(round(time.time()-tmps5,2),' secondes')
    
if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 3 fois. \n')
    print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' a échoué. Les modèles ne sont pas sauvegardés')
    print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
    tmps2=round(time.time()-tmps55,2)
    print ("Job done in = %f" %tmps2,'seconds')

if (precision_up*100) >= 69 and (precision_down*100) >= 69:
    save_model(model_up, model_down)

    print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' a été effecué avec succès. Les modèles ont été sauvegardés')
    print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
    tmps2=round(time.time()-tmps55,2)
    print ("Job done in = %f" %tmps2,'seconds')
    

 
Deeping in blue for  USDJPY_1H 
2626/2626 [==============================] - 0s 15us/step
560.26  secondes

Ticker :  USDJPY_1H Precision Up : 0.0  ---- Precision Down 6.382978723404255 

Test Raté 1 fois. On remet cela. 

2626/2626 [==============================] - 0s 15us/step
544.76  secondes

Ticker :  USDJPY_1H Precision Up : 13.793103448275861  ---- Precision Down 5.263157894736842 

Test Raté 2 fois. On remet cela. 



KeyboardInterrupt: 